# ReInvent 2019 Amazon Personalize Workshop

![MainDiagram](static/imgs/image.png)

## Agenda

At a high level working with Amazon Personalize follows the steps in the diagram below:

![FlowDiagram](static/imgs/personalize_process.png)

The specific process that you will be following is:

1. Imports and Setup
1. Preparing Your Data
1. Importing Your Data
1. Selecting a Recipe
1. Training a Solution
1. Deploying a Campaign
1. Getting Recommendations
1. Real-Time Interactions
1. Next Steps

In each of these steps you'll see and execute code snippets written in Python using our Boto3 SDK, these snippets can be modified to be components of your production integration with Personalize. 

This notebook will walk you through the steps to build a recommendation model for movies that are tailored to specific users, the goal is to recommend movies based on that users' history of positive interactions with movies.

The data is provided via the [MovieLens Project](https://movielens.org), you can read more about it later if you are interested.

The content below has been written to guide you through the process within the timeline of a workshop session. You can deploy the CloudFormation template you found on GitHub later to your own account and can walk through notebooks 1-3 to do the same exercise again. Additionally you can update the content in the cells to reflect your own data and it is an effective path to building a custom recommendaiton model for your use case.

## How to Use the Notebook

Code is broken up into cells like the one below. There's a triangular `Run` button at the top of this page you can click to execute each cell and move onto the next, or you can press `Shift` + `Enter` while in the cell to execute it and move onto the next one.

As a cell is executing you'll notice a line to the side showcase an `*` while the cell is running or it will update to a number to indicate the last cell that completed executing after it has finished exectuting all the code within a cell.


Simply follow the instructions below and execute the cells to get started with Amazon Personalize.



## Imports and Setup 

Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like boto3(The AWS SDK) and Pandas/Numpy which are core data science tools. Teh cell below will import them for use here.

In [2]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime
import uuid

Next you will want to validate that your environment can communicate successfully with Amazon Personalize, the lines below do just that.

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client(service_name='personalize-events')

## Preparing Your Data

To begin you will need a collection of data points where users have interacted with content in some way. Amazon Personalize assumes that if an interaction is recorded it is a positive one, you will see how we use that later. 

The cell below will download the data you need, extract the content from a Zip file, and then display a small portion of it to your screen.

In [4]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

--2019-11-26 23:04:45--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘ml-100k.zip’ not modified on server. Omitting download.

Archive:  ml-100k.zip
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflat

,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,196,242,3,881250949
1,186,302,3,891717742
...,...,...,...,...
99998,13,225,2,882399156
99999,12,203,3,879959583


As you can see the data contains a UserID, ItemID, Rating, and Timestamp.

We are now going to remove the entries with low rankings, and remove the column `RATING` before importing the data into Personalize.

This is done so that the system learns from positive interactions, that is how the algoritihms provided in Personalize work best. As new approaches come to market that guidance could shift.

Once the data has been prepared you will save it to a CSV file locally with the last line.

In [5]:
data = data[data['RATING'] > 3]                # Keep only movies rated higher than 3 out of 5.
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
filename = "movie-lens-100k-prepared.csv"
data.to_csv(filename, index=False)

Before you can upload the data into S3 you will need to create a bucket, the cell below will do that.

In [6]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "reinventforecastworkshop"
print(bucket_name)
s3.create_bucket(Bucket=bucket_name)

059124553121reinventforecastworkshop


Now you can upload the file to S3 and it will be ready to import into Amazon Personalize.

In [8]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(filename).upload_file(filename)

## Importing Your Data

The steps for importing your data are:

1. Create a Dataset Group.
1. Determine the schema for your dataset.
1. Create a Dataset using your schema.
1. Run an ImportJob to load the data for use with Personalize.


In Amazon Personalize a Dataset Group is how your information is isolated from any other experiment. No information is shared between these groups at all. A Dataset Group can contain your interaction data, item metadata, user metadata, event trackers, solutions, and campaigns. To learn more about them: https://docs.aws.amazon.com/personalize/latest/dg/API_DatasetGroup.html 

Begin by creating a Dataset Group

#### Create Dataset Group

In [9]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-RI-demo"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:059124553121:dataset-group/personalize-RI-demo",
  "ResponseMetadata": {
    "RequestId": "08ce8d45-9d11-4cbd-bb74-0a4435bc1c90",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 25 Nov 2019 22:32:18 GMT",
      "x-amzn-requestid": "08ce8d45-9d11-4cbd-bb74-0a4435bc1c90",
      "content-length": "98",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Dataset Group to Have ACTIVE Status


Wait for the Dataset Group to be active before moving to the next step, the loop below will let you know when the Dataset Group is active. This should take no more than 1 minute to complete.

In [10]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(5)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


#### Create Schema


Now you can determine the Schema for your Dataset. In this workshop time is limited so you will only be utilizing user-item-interaction data or interaction data for short. The cell below contains the required items for this dataset and maps to the structure of the CSV that you uploaded earlier.

In [11]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

The cell below will create a schema inside Amazon Personalize that can be connected to a Dataset this is how Personalize understands the content within your CSV file.

In [13]:
create_schema_response = personalize.create_schema(
    name = "personalize-ri-demo-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:059124553121:schema/personalize-ri-demo-schema",
  "ResponseMetadata": {
    "RequestId": "23cded65-0cf9-4beb-a6df-8200ca7fdfcd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 25 Nov 2019 22:39:20 GMT",
      "x-amzn-requestid": "23cded65-0cf9-4beb-a6df-8200ca7fdfcd",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create Dataset


Next create the Dataset for interactions and assign it the Schema provided and assign it to the Dataset Group you created earlier with this code:

In [14]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-ri-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:059124553121:dataset/personalize-RI-demo/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "801a9eb3-d88b-4e3d-a4a9-536e78a783c8",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 25 Nov 2019 22:39:22 GMT",
      "x-amzn-requestid": "801a9eb3-d88b-4e3d-a4a9-536e78a783c8",
      "content-length": "100",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Attach Policy to S3 Bucket

Before you can execute the import job for your data you will need to attach a bucket policy to your S3 bucket to allow Personalize to communicate with it, as well as an IAM role for the service to use within this AWS account.

In [71]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '15754AFE12D4D851',
  'HostId': 'iq+fLMF8Gfr8+3PuXD9YZUQNkmA4uytwyIUzvzfYsu91SKoqGCy+5J5anPWucjxPVqirYEmRuoY=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'iq+fLMF8Gfr8+3PuXD9YZUQNkmA4uytwyIUzvzfYsu91SKoqGCy+5J5anPWucjxPVqirYEmRuoY=',
   'x-amz-request-id': '15754AFE12D4D851',
   'date': 'Tue, 26 Nov 2019 16:49:28 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

Just a note, IAM changes can take just a few seconds to be confirmed so this section will take 1 minute to complete.

#### Create Personalize Role

In [18]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleRIDemo"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::059124553121:role/PersonalizeRoleRIDemo


At last you are ready to import the data into Personalize, the first cell below will start the process and the second contains a while loop that will poll the service to determine when the import job has completed. 

During this workshop you are importing a relatively small dataset and it may seem like this takes a while for such a small file. The leading use of time here is provisioning dedicated resources that will actually run the task. This allows Personalize to provide HIPAA compliance for example, it also means that just because your files are larger that it won't take an extremely long time, once the resources are up and running the import is pretty quick.

It may take up to 20 minutes for the import process to complete. 

#### Create Dataset Import Job

In [20]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-ri-import",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:059124553121:dataset-import-job/personalize-ri-import",
  "ResponseMetadata": {
    "RequestId": "7d533ede-60a3-4daa-9cc4-6088dd226d8d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 25 Nov 2019 22:41:05 GMT",
      "x-amzn-requestid": "7d533ede-60a3-4daa-9cc4-6088dd226d8d",
      "content-length": "109",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Dataset Import Job to Have ACTIVE Status

In [21]:
current_time = datetime.datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  10:41:07 PM
DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
Import Completed on:  10:53:08 PM


## Selecting a Recipe

Inside Personalize the algorithms available are called recipes, the code below will give you a full list of them. They each have various use cases and detailed information can be found here: https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html

In this workshop you will be using HRNN(Hierarchical Recurrent Neural Network) quoting the docs:

```
HRRN is a hierarchical recurrent neural network, which can model the user-item interactions across a given timeframe. Use the HRNN recipe when user behavior changes over time, which is referred to the evolving intent problem.

To train a model, HRNN uses the Interactions dataset from a dataset group. A dataset group is a set of related datasets, which can include the Users, Items, and Interactions datasets.
```

A paper explaining it more in detail as it relates to recommendations: https://openreview.net/pdf?id=ByzxsrrkJ4 


It is used here as an example of a recommendation system built using deep neural networks and can be trained using only our interactions data. This is a great place to start when running your own experiments later as well.

In [22]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 65000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 64000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 64000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   'stat

The cell below selects the `HRNN` recipe.

In [23]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

## Training a Solution

Within Amazon Personalize a model trained on customer data is called a Solution, these are versioned and so you will first create a Solution, then a Solution Version. The versions are utilized to track model improvement over time based on newer available data. 

Creating a Solution itself is nearly instantaneous however the actual training to create a version can take a bit of time. This is usually the longest waiting period in the process. If you are doing this outside of a workshop it is a great time to check your emails, grab a coffee, etc.

#### Create Solution

In [24]:
create_solution_response = personalize.create_solution(
    name = "personalize-ri-soln-hrnn",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:059124553121:solution/personalize-ri-soln-hrnn",
  "ResponseMetadata": {
    "RequestId": "30e8563e-6227-445e-86c8-5d4655ba6fd1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 25 Nov 2019 22:54:06 GMT",
      "x-amzn-requestid": "30e8563e-6227-445e-86c8-5d4655ba6fd1",
      "content-length": "94",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create Solution Version

In [25]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:059124553121:solution/personalize-ri-soln-hrnn/f7649797",
  "ResponseMetadata": {
    "RequestId": "3d8a5d1c-307b-4e94-ad2a-5d2f4fcba9ca",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 25 Nov 2019 22:54:07 GMT",
      "x-amzn-requestid": "3d8a5d1c-307b-4e94-ad2a-5d2f4fcba9ca",
      "content-length": "110",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Solution Version to Have ACTIVE Status

This will take at least 20 minutes.

In [26]:
current_time = datetime.datetime.now()
print("Training Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Training Completed on: ", current_time.strftime("%I:%M:%S %p"))

Training Started on:  10:54:09 PM
SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRES

#### Get Metrics of Solution Version

Now that your solution and version exists, you can obtain the metrics for it to judge its performance. These metrics are not particularly good as it is a demo set of data, but with larger more compelx datasets you should see improvements.

In [27]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:059124553121:solution/personalize-ri-soln-hrnn/f7649797",
  "metrics": {
    "coverage": 0.3122,
    "mean_reciprocal_rank_at_25": 0.0416,
    "normalized_discounted_cumulative_gain_at_10": 0.0442,
    "normalized_discounted_cumulative_gain_at_25": 0.0575,
    "normalized_discounted_cumulative_gain_at_5": 0.0408,
    "precision_at_10": 0.0056,
    "precision_at_25": 0.0045,
    "precision_at_5": 0.009
  },
  "ResponseMetadata": {
    "RequestId": "b56bd094-f453-4a68-a40a-d43501945a16",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 25 Nov 2019 23:41:28 GMT",
      "x-amzn-requestid": "b56bd094-f453-4a68-a40a-d43501945a16",
      "content-length": "407",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Create and Wait for the Campaign

Now that you have a working solution version you will need to create a campaign to use it with your applications. A campaign is simply a hosted copy of your model. Again there will be a short wait so after executing you can take a quick break while the infrastructure is being provisioned.

#### Create Campaign

In [28]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-ri-camp",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:059124553121:campaign/personalize-ri-camp",
  "ResponseMetadata": {
    "RequestId": "90250494-6265-4e51-9ef3-bd3ed29a4d6e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 25 Nov 2019 23:41:31 GMT",
      "x-amzn-requestid": "90250494-6265-4e51-9ef3-bd3ed29a4d6e",
      "content-length": "89",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Campaign to Have ACTIVE Status

In [30]:
current_time = datetime.datetime.now()
print("Deploying Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Deploying Completed on: ", current_time.strftime("%I:%M:%S %p"))

Deploying Started on:  03:10:41 AM
Campaign: ACTIVE
Deploying Completed on:  03:10:41 AM


## Get Sample Recommendations

After the campaign is active you are ready to get recommendations. First we need to select a random user from the collection. Then we will create a few helper functions for getting movie information to show for recommendations instead of just IDs.

In [31]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

USER: 561


In [8]:
# First load items into memory
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    return items.iloc[movie_id]['TITLE']


#### Select a User and an Item

#### Call GetRecommendations

The code below will get recommendations for the random user selected, it then places the recommendations in a dataframe and renders it.

In [87]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['Original Recommendations'])
recommendations_df

Recommendations for user:  579


,Original Recommendations
0,Blade Runner (1982)
1,Go Fish (1994)
2,Jane Eyre (1996)
3,Batman (1989)
4,"Brothers McMullen, The (1995)"
5,Snow White and the Seven Dwarfs (1937)
6,Flipper (1996)
7,Shadowlands (1993)
8,Casino (1995)
9,"Blob, The (1958)"


## Creating an Event Tracker

Before your recommendation system can respond to real time events you will need an event tracker, the code below will generate one and can be used going forward with this lab. Feel free to name it something more clever.

In [34]:
response = personalize.create_event_tracker(
    name='MovieClickTrackerRI',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

arn:aws:personalize:us-east-1:059124553121:event-tracker/2a0e0f0d
4a7b3986-d09f-4971-85d9-d3e5685d75d9


In [35]:
event_tracker_arn = response['eventTrackerArn']

## Simulating User Behavior

The lines below provide a code sample that simulates a user interacting with a particular item, you will then get recommendations that differ from those when you started.

In [36]:
session_dict = {}

In [38]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
    )

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the title of the movie for the header of the column
    movie_title_clicked = get_movie_title(movie_to_click)
    # Interact with the movie
    send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)
    # Sleep for 2 seconds
    time.sleep(2)
    # Get new recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        title = get_movie_title(item['itemId'])
        recommendation_list.append(title)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_title_clicked])
    # Add this dataframe to the old one
    recommendations_df = recommendations_df.join(new_rec_DF)
    return recommendations_df

The cell below will execute interactions and update the dataframe with the results so you can see the overall impact. Feel free to run the cell several times with movie IDs less than 1000. Note for this demo it will error if you use the same movie twice, this is due to how we showcase the headers.

In [44]:
movie_to_click = 180
recommendations_df = get_new_recommendations_df(recommendations_df, movie_to_click)
recommendations_df

,OriginalRecs,Bedknobs and Broomsticks (1971),Lone Star (1996),"Lawnmower Man, The (1992)",Return of the Jedi (1983)
0,Apocalypse Now (1979),Apocalypse Now (1979),Thinner (1996),Wes Craven's New Nightmare (1994),Wes Craven's New Nightmare (1994)
1,"Blob, The (1958)","Blob, The (1958)","Blob, The (1958)","Brothers McMullen, The (1995)",Heathers (1989)
2,Laura (1944),Bonnie and Clyde (1967),"Brothers McMullen, The (1995)",Kolya (1996),Wolf (1994)
3,"Wizard of Oz, The (1939)","Omen, The (1976)",Sleeper (1973),Executive Decision (1996),"Blues Brothers, The (1980)"
4,Notorious (1946),Jeffrey (1995),Bonnie and Clyde (1967),Star Trek IV: The Voyage Home (1986),"Fish Called Wanda, A (1988)"
5,Forbidden Planet (1956),"Abyss, The (1989)","Omen, The (1976)",Miller's Crossing (1990),Executive Decision (1996)
6,"Day the Earth Stood Still, The (1951)",Raging Bull (1980),"Man Who Would Be King, The (1975)",Sleeper (1973),"Shining, The (1980)"
7,"Streetcar Named Desire, A (1951)","Silence of the Lambs, The (1991)",Gone with the Wind (1939),Thinner (1996),Cat on a Hot Tin Roof (1958)
8,The Innocent (1994),Paths of Glory (1957),"Paris, Texas (1984)",Welcome to the Dollhouse (1995),"Graduate, The (1967)"
9,Glory (1989),"Mirror Has Two Faces, The (1996)",Home Alone (1990),Wolf (1994),12 Angry Men (1957)


## Batch Recommendations

So far you have seen how to generate recommendations via an API call and interact with event trackers. That works well for many applications but you may find yourself wanting to cache all recommendations for users locally or even to study the recommendations for new ideas. To support that Amazon Personalize supports batch exporting of your recommendations to a file as well. The cells below will walk you through sending recommendations to a file in S3 and then will show its contents. The file can be downloaded from S3 to your local computer as well.

First you will need to create a JSON file of user IDS

In [10]:
user_IDs = ['561', '233', '579']

json_input_filename = "json_input.json"
with open(json_input_filename, 'w') as json_input:
    for user_id in user_IDs:
        json_input.write('{"userId": "' + user_id + '"}\n')

Now upload this file to S3:

In [11]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://059124553121reinventforecastworkshop/json_input.json


Now that the input file is available in S3, you need to define where the output will go, and create a batch inference job.

In [69]:
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://059124553121reinventforecastworkshop/


The cell below creates the batch job.

In [91]:
personalize_rec = boto3.client(service_name='personalize')
batchInferenceJobArn = personalize_rec.create_batch_inference_job (
    solutionVersionArn = solution_version_arn,
    jobName = "RI-Workshop-Batch-Export-Job",
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

The next cell will poll until the export has completed.

In [ ]:
current_time = datetime.datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize_rec.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

With the data successfully exported, grab the file and parse it.

In [29]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the movies
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            title = get_movie_title(item)
            recommendation_list.append(title)
        if 'recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            recommendations_df = recommendations_df.join(new_rec_DF)
        else:
            recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
recommendations_df

,User: 561,User: 233,User: 579
0,Apocalypse Now (1979),Heat (1995),Blade Runner (1982)
1,"Blob, The (1958)","Full Monty, The (1997)",Go Fish (1994)
2,Laura (1944),Evita (1996),Jane Eyre (1996)
3,"Wizard of Oz, The (1939)",U Turn (1997),Batman (1989)
4,Notorious (1946),Tomorrow Never Dies (1997),"Brothers McMullen, The (1995)"
5,Forbidden Planet (1956),"Replacement Killers, The (1998)",Snow White and the Seven Dwarfs (1937)
6,"Day the Earth Stood Still, The (1951)",Ulee's Gold (1997),Flipper (1996)
7,"Streetcar Named Desire, A (1951)",3 Ninjas: High Noon At Mega Mountain (1998),Shadowlands (1993)
8,The Innocent (1994),Jungle2Jungle (1997),Casino (1995)
9,Glory (1989),As Good As It Gets (1997),"Blob, The (1958)"


In the cell above you can see the various recommendations for the users provided, in a real scenario the list of users could be your entire userbase allowing you to quickly reference and compare results between them.

## Conclusion

In this workshop you successfully created a dataset group, a dataset based on interactions, trained a recommendation model based on the data, deployed a campaign to generate recommendations, evaluated the initial recommendations, leveraged real time event tracking for even better recommendations, and concluded with a bulk export of your recommendations to a standard file. 

This content will stay public on GitHub and can be used within your organization in order to build custom recommendation models, take a look at the various other notebooks in the future.

Thank you, and good luck in your next Machine Learning project!